In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
%matplotlib inline 

In [2]:
churn_df = pd.read_csv('churn.csv')
col_names = churn_df.columns.tolist()
to_show = col_names[:6] + col_names[-6:]
churn_df[to_show].head(6)
churn_result = churn_df['Churn?']
y = np.where(churn_result == 'True.',1,0)
to_drop = ['State','Area Code','Phone','Churn?']
churn_feat_space = churn_df.drop(to_drop,axis=1)
yes_no_cols = ["Int'l Plan","VMail Plan"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'yes'
features = churn_feat_space.columns
print(features)

Index(['Account Length', 'Int'l Plan', 'VMail Plan', 'VMail Message',
       'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls',
       'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins',
       'Intl Calls', 'Intl Charge', 'CustServ Calls'],
      dtype='object')


spike_cols = [col for col in df.columns if 'Int' in col]

churn_feat_space=churn_feat_space['Account Length',spike_cols[0],'VMail Plan','VMail Message','Day Calls','Day Charge']

'Account Length', u'Int'l Plan', u'VMail Plan', u'VMail Message', u'Day Mins', u'Day Calls', u'Day Charge', u

In [5]:
X = churn_feat_space.as_matrix().astype(np.float)

scalar= StandardScaler()
X= scalar.fit_transform(X)
print(X.shape)
print(np.unique(y))

(3333, 17)
[0 1]


In [52]:
def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=3,shuffle=True)
    y_pred = y.copy()
    
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [53]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import average_precision_score

In [54]:
def accuracy(y_true,y_pred):
    return np.mean(y_true==y_pred)

In [55]:
print("Logistic R",accuracy(y, run_cv(X,y,LR)))
print("KNN",accuracy(y, run_cv(X,y,KNN)))
print("Random CL",accuracy(y, run_cv(X,y,RF)))
print("GBC",accuracy(y, run_cv(X,y,GBC)))
print("Logistic R",accuracy(y, run_cv(X,y,LR)))

Logistic R 0.858085808581
KNN 0.892289228923
Random CL 0.940894089409
GBC 0.950195019502
Logistic R 0.859285928593
